In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [ ]:
urlstart = 'https://www.imdb.com/search/title/?title_type=feature&release_date='
#https://www.imdb.com/search/title/?title_type=feature has all movies ever
#the link only shows 50 movies at a time. We need to split it up to have less than 50

urlend = '&sort=num_votes,desc&num_votes=100000,&languages=en'
#There are hundreds of thousands of small budget movies. We will make sure 100000 people have reviwed a movie and it is in english to start

monthstartlist = ['-01-01','-02-01','-03-01','-04-01','-05-01','-06-01','-07-01','-08-01','-09-01','-10-01','-11-01','-12-01']
monthstartlist = monthstartlist[::-1]
monthendlist = ['-01-31','-02-28','-03-31','-04-30','-05-31','-06-30','-07-31','-08-31','-09-30','-10-31','-11-30','-12-31']
monthendlist = monthendlist[::-1]
#we will split up the movies month by month to get less than 50 on the page. List is backwards in time. If I have time I will update this for leap years.

yearlist = []
for i in range(31):
    yearlist.append(str(2022-i))
#we will examine the last 30 years for now

urllist = []
for i in yearlist:
    for fdx,f in enumerate(monthstartlist):
        urllist.append(urlstart+i+f+','+i+monthendlist[fdx]+urlend)
#now we have a series of links with all relevant movies in the last 30 years. Parameters can be updated easily if needed

In [ ]:
USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:65.0) Gecko/20100101 Firefox/65.0"
headers = {"user-agent": USER_AGENT} # adding the user agent

#We need a user agent to get access. This is from stackoverflow.

In [ ]:
namelist = []
imdbratingslist = []
imdbscorelist = []
contentratinglist = []
releasedlist = []
genreslist = []
general_keywordslist = []
starslist = []
directorslist = []
writerslist = []
runtimelist = []
awardslist = []
nomslist = []
budgetlist = []
budgetclist = []
grossnalist = []
openingnalist = []
grosswolist = []
errorlog = []
#Initiallize lists. We will start from lists because we have to append in iterations. These will be dataframe columns

for URL in urllist:
    
    resp = requests.get(URL, headers=headers)
    bigsoup = BeautifulSoup(resp.content, "html.parser")
    result_set = bigsoup.find_all('h3',class_='ipc-title__text')
    #This gets us every name of a movie on the page
    
    for g in result_set:
        name = str(g).split(" ", 2)[2][:-5]
        if name != 'viewed':
            try:
                link = 'https://www.imdb.com/'+re.split('href="|" tabindex',str(bigsoup.find_all('a',class_="ipc-title-link-wrapper")).split(name)[0])[-2]
                #Extracts the link within the link for the movies page
                
                soup = BeautifulSoup(requests.get(link, headers=headers).content, "html.parser")
                ar,genre,released,keywords,tdur,stars,directors,writers,runtime = re.split(',"aggregateRating":|,"genre":|,"datePublished":|,"keywords":|,"actor":|,"director":|,"creator":|,"duration|⭐',str(soup))[1:10]
                arlist = re.split(',"|:|}',ar)
                imdbratings = arlist[3]
                imdbscore = arlist[9]
                contentrating = arlist[12].strip('"')
                released = released.strip('"')
                genres = genre[1:-1].split(',')
                genres = [s.strip('"') for s in genres]
                general_keywords = keywords.split(':{')[0].split(',')
                general_keywords = [s.strip('"') for s in general_keywords]
                stars = stars.split('name":"')[1:]
                stars =  [s.split('"}')[0] for s in stars]
                directors = directors.split('name":"')[1:]
                directors = [s.split('"}')[0] for s in directors]
                writers = writers.split('name":"')[1:]
                writers = [s.split('"}')[0] for s in writers]
                runtime = re.split('PT|"}',runtime)[1]
                runtime = re.split('H|M',runtime)
                runtime = 60*int(runtime[0])+int(runtime[1])
                #The metadata in the summary section cannot be searched with soup.find so I used text splitting
                
                aftermath = soup.find_all('span',class_="ipc-metadata-list-item__list-content-item")
                if 'win' in str(aftermath[0]):
                    awards = str(aftermath[0])
                    awards = re.split('>| win',awards)[1]
                    noms = str(aftermath[0])
                    noms = re.split('&amp; | nominati',noms)[1]
                else:
                    awards = 0
                    noms = re.split('>| nominati',str(aftermath[0]))[1]
                if len(aftermath) > 7 or ((len(aftermath) > 6) and (':' not in str(aftermath[-1]))):
                    budget = str(aftermath[2])
                    budget = re.split('>| \(est',budget)[1]
                    budgetcurrency = budget[0]
                    budgetnum = budget[1:].replace(',','')
                    grossna = str(aftermath[3])
                    grossna = re.split('>|<',grossna)[2]
                    grossna = grossna[1:].replace(',','')
                    openingna = str(aftermath[4])
                    openingna = re.split('>|<',openingna)[2]
                    openingna = openingna[1:].replace(',','')
                    grosswo = str(aftermath[6])
                    grosswo = re.split('>|<',grosswo)[2]
                    grosswo = grosswo[1:].replace(',','')
                else:
                    budgetcurrency = 'unknown'
                    budgetnum = 'unknown'
                    grossna = 'unknown'
                    openingna = 'unknown'
                    grosswo = 'unknown'
                print(name)
                #This is data for initial analysis. We need more detailed info from boxofficemojo or imdbpro later becuase its a little flawed here. Will take this out when done
                
                namelist.append(name)
                imdbratingslist.append(imdbratings)
                imdbscorelist.append(imdbscore)
                contentratinglist.append(contentrating)
                releasedlist.append(released)
                genreslist.append(genres)
                general_keywordslist.append(general_keywords)
                starslist.append(stars)
                directorslist.append(directors)
                writerslist.append(writers)
                runtimelist.append(runtime)
                awardslist.append(awards)
                nomslist.append(noms)
                budgetlist.append(budgetnum)
                budgetclist.append(budgetcurrency)
                grossnalist.append(grossna)
                openingnalist.append(openingna)
                grosswolist.append(grosswo)
                #append everything to lists if nothing went wrong
            except:
                print(name+'didntwork')
                errorlog.append(name)
                #will handle these later

In [ ]:
print(len(namelist),len(grosswolist))
#We have problems when these are not equal

In [ ]:
df = pd.DataFrame(list(zip(namelist,imdbratingslist,imdbscorelist,contentratinglist,releasedlist,genreslist,general_keywordslist,starslist,directorslist,writerslist,runtimelist,nomslist,awardslist,budgetlist,budgetclist,grossnalist,openingnalist,grosswolist)),columns =['Name', 'IMDB Ratings', 'IMDB Score', 'Content Rating', 'Date Released', 'Genres', 'Keywords', 'Stars', 'Directors', 'Writers', 'Runtime', 'Nominations', 'Awards', 'Estimated Budget', 'Budget Currency', 'Gross Revenue US&Canada USD', 'Opening Weekend Revenue US&Canada USD', 'Gross Revenue Worldwide USD'])
df.to_csv('Prophet1.csv', index=False)
#save